In [1]:
import pandas as pd
import smtplib
import time
from email.mime.text import MIMEText
import threading


In [2]:
# Google Sheet CSV link (from publish to web)
CSV_URL = "https://docs.google.com/spreadsheets/d/e/2PACX-1vRJekzWK645i0iAjs4s1pS6vnExHsOnrSGBXnA7EjUbm3w7-rCFPdqvhqJ8IO9xjykBSV40WR05pDZU/pub?output=csv"

# Email config
GMAIL_USER = "fake.avidedania@gmail.com"
GMAIL_PASSWORD = "umrp bjbu bsul dhgy"
TO_EMAIL = "social.avidedania@gmail.com"


In [3]:
def send_email_notification(subject, body):
    msg = MIMEText(body)
    msg['Subject'] = subject
    msg['From'] = GMAIL_USER
    msg['To'] = TO_EMAIL

    try:
        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls()
            server.login(GMAIL_USER, GMAIL_PASSWORD)
            server.sendmail(GMAIL_USER, TO_EMAIL, msg.as_string())
        print("✅ Email sent!")
    except Exception as e:
        print("❌ Email failed:", e)


In [4]:
def get_sheet_row_count():
    try:
        df = pd.read_csv(CSV_URL)
        return len(df), df
    except Exception as e:
        print("❌ Failed to read sheet:", e)
        return -1, None


In [5]:
prev_count, prev_df = get_sheet_row_count()
print(f"Initial rows: {prev_count}")

def monitor_sheet(interval=10):
    global prev_count, prev_df
    print("🔍 Monitoring started... Press Stop in Jupyter to interrupt.")
    try:
        while True:
            time.sleep(interval)
            curr_count, curr_df = get_sheet_row_count()

            if curr_count != prev_count:
                if curr_count > prev_count:
                    # A row was added
                    added_row = curr_df.iloc[-1]  # Assume new row is at the end
                    subject = "🚨 Row Added"
                    body = f"A new row was added:\n\n{added_row.to_string()}"
                    send_email_notification(subject, body)

                elif curr_count < prev_count:
                    # A row was removed
                    removed_rows = prev_df[~prev_df.apply(tuple, 1).isin(curr_df.apply(tuple, 1))]
                    if not removed_rows.empty:
                        subject = "⚠️ Row Removed"
                        body = f"A row was removed:\n\n{removed_rows.iloc[0].to_string()}"
                        send_email_notification(subject, body)

                prev_count, prev_df = curr_count, curr_df
    except KeyboardInterrupt:
        print("⛔ Monitoring stopped manually.")


Initial rows: 12250


In [6]:
thread = threading.Thread(target=monitor_sheet)
thread.start()


🔍 Monitoring started... Press Stop in Jupyter to interrupt.
✅ Email sent!
✅ Email sent!
✅ Email sent!
✅ Email sent!
✅ Email sent!
✅ Email sent!
✅ Email sent!
✅ Email sent!
